In [1]:
import numpy as np
import pandas as pd

In [2]:
train= pd.read_csv('data/training.csv', delimiter=';', decimal=',')

In [3]:
train.sample(3)

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14,v15,v16,v17,classLabel
1986,b,27.83,0.000154,u,-4.415589,0.969767,3.750,t,t,5,t,g,100.0,3,1000000.0,NaN,1,yes.
950,a,44.17,0.000666,u,-1.789231,0.820354,7.375,t,t,3,t,g,0.0,0,0.0,t,1,yes.
2507,b,34.17,0.000154,u,2.405765,0.791931,1.540,t,t,1,t,g,520.0,50000,5200000.0,NaN,1,yes.


# Extending pandas clas
[Reference for inheriting pandas](https://pandas.pydata.org/pandas-docs/stable/development/extending.html)



In [4]:
@pd.api.extensions.register_dataframe_accessor("hlst")
class HighLevelStats():
    def __init__(self, pObject):
        self._pObject = pObject
    
    @property    
    def print_stats_numeric(self):
        return self._pObject.describe(include= np.number).transpose()
    
    @property    
    def print_stats_object(self):
        return self._pObject.describe(include= np.object).transpose()

In [5]:
train.hlst.print_stats_numeric

,count,mean,std,min,25%,50%,75%,max
v2,3661.0,3.282071e+01,1.266618e+01,13.750000,23.000000,2.867000e+01,4.083000e+01,8.025000e+01
v3,3700.0,5.847872e-04,5.402007e-04,0.000000,0.000150,4.250000e-04,9.625000e-04,2.800000e-03
v5,3700.0,-5.331808e-02,2.871240e+00,-4.996939,-2.516349,-5.278621e-02,2.406251e+00,4.999274e+00
v6,3700.0,7.841879e-01,1.365138e-01,0.273945,0.693130,7.878424e-01,8.760522e-01,1.215066e+00
v7,3700.0,3.439496e+00,4.335229e+00,0.000000,0.500000,1.750000e+00,5.000000e+00,2.850000e+01
v10,3700.0,4.160000e+00,6.750553e+00,0.000000,0.000000,2.000000e+00,6.000000e+00,6.700000e+01
v13,3600.0,1.626950e+02,1.560457e+02,0.000000,0.000000,1.200000e+02,2.800000e+02,1.160000e+03
v14,3700.0,2.246706e+03,8.708571e+03,0.000000,0.000000,1.130000e+02,1.059750e+03,1.000000e+05
v15,3600.0,1.626950e+06,1.560457e+06,0.000000,0.000000,1.200000e+06,2.800000e+06,1.160000e+07
v17,3700.0,9.254054e-01,2.627716e-01,0.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00


In [6]:
train.hlst.print_stats_object

,count,unique,top,freq
v1,3661,2,b,2382
v4,3636,3,u,3055
v8,3700,2,t,3171
v9,3700,2,t,2239
v11,3700,2,f,1924
v12,3700,3,g,3430
v16,1555,2,t,1023
classLabel,3700,2,yes.,3424


# Tensorflow StatGen maybe better